# 🎭 第一章 练习 3: 混淆偏差 (Confounding Bias)

---

## 让偏差「现形」

在前两个练习中，我们知道了：
- 潜在结果框架告诉我们「什么是因果效应」
- 因果图告诉我们「为什么会有偏差」

但一个关键问题还没回答：**偏差到底有多大？能量化吗？**

### 一个让你崩溃的场景 😱

你是一名医学研究者，发现了一个惊人的结论：

> "喝红酒的人比不喝酒的人心脏病发病率低 30%！"

新闻标题已经想好了：*《科学证明：红酒是心脏的保护神》*

但等等...你的同事问了几个问题：

- 喝红酒的人是不是收入更高？（能买得起红酒）
- 收入高的人是不是医疗条件更好？
- 那这 30% 的差异有多少是红酒的功劳，多少是收入的功劳？

这就是**混淆偏差**的可怕之处——它会让无效的治疗看起来有效，也会让有效的治疗看起来无效！

---

## 📚 本节学习目标

1. 理解混淆偏差的数学公式
2. 学会量化偏差的大小
3. 深入理解 Simpson's Paradox
4. 掌握敏感性分析方法

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！")

---

## 🧮 Part 1: 混淆偏差公式 (Omitted Variable Bias)

### 核心公式

当我们遗漏了一个混淆变量 X，朴素估计和真实效应之间的偏差为：

$$\text{Bias} = \gamma \times \delta$$

其中：
- $\gamma$：混淆变量 X 对结果 Y 的效应（控制 T 后）
- $\delta$：混淆变量 X 与处理 T 的关联

### 直觉理解

偏差 = (X 对 Y 的影响) × (X 与 T 的关联)

**两个乘数都不为零时，才会有偏差！**

| 情况 | γ (X→Y) | δ (X-T) | 偏差 | 例子 |
|-----|---------|---------|------|------|
| 无偏差 | 0 | 任意 | 0 | X 不影响 Y |
| 无偏差 | 任意 | 0 | 0 | X 与 T 无关 |
| 正向偏差 | + | + | + | 高估效应 |
| 正向偏差 | - | - | + | 高估效应 |
| 负向偏差 | + | - | - | 低估效应 |
| 负向偏差 | - | + | - | 低估效应 |

### 🔬 动手验证偏差公式

In [ ]:
# 首先，让我们生成一些模拟数据
np.random.seed(42)
n = 2000

# DAG: X → T, X → Y, T → Y
# X 是混淆变量

# 生成混淆变量 X
X = np.random.randn(n)

# 生成处理 T（受 X 影响）
# delta = 1.5（X 对 T 的影响系数）
T = (np.random.randn(n) + 1.5 * X > 0).astype(int)

# 生成结果 Y（受 T 和 X 影响）
# 真实 ATE = 2, gamma = 1.5
true_ate = 2.0
gamma_true = 1.5
Y = 5 + true_ate * T + gamma_true * X + np.random.randn(n) * 0.5

df = pd.DataFrame({'X': X, 'T': T, 'Y': Y})

print("📊 数据概览:")
print(f"   样本量: {n}")
print(f"   真实 ATE: {true_ate}")
print(f"   X 对 Y 的真实效应 (γ): {gamma_true}")
print(f"   处理组比例: {T.mean():.2%}")
display(df.head())

In [ ]:
def calculate_confounding_bias(df: pd.DataFrame, confound_var: str = 'X') -> dict:
    """
    计算混淆偏差的各个组成部分
    
    Omitted Variable Bias 公式: bias = γ × δ
    
    - γ (gamma): 混淆变量 X 对结果 Y 的效应（控制 T 后）
    - δ (delta): 混淆变量 X 与处理 T 的关联
    """
    # 朴素估计（不控制 X）
    model_naive = LinearRegression()
    model_naive.fit(df[['T']], df['Y'])
    naive_estimate = model_naive.coef_[0]
    
    # 调整估计（控制 X）
    model_adjusted = LinearRegression()
    model_adjusted.fit(df[['T', confound_var]], df['Y'])
    adjusted_estimate = model_adjusted.coef_[0]
    
    # TODO: 计算 gamma（X 对 Y 的效应，控制 T）
    # 提示: 这是 model_adjusted 中 X 的系数
    gamma = None  # 👈 你的代码: model_adjusted.coef_[1]
    
    # TODO: 计算 delta（X 与 T 的关联）
    # 回归 T ~ X，取 X 的系数
    model_delta = LinearRegression()
    model_delta.fit(df[[confound_var]], df['T'])
    delta = None  # 👈 你的代码: model_delta.coef_[0]
    
    # TODO: 计算理论偏差
    theoretical_bias = None  # 👈 你的代码: gamma * delta
    
    # 实际偏差（从估计值直接算）
    actual_bias = naive_estimate - adjusted_estimate
    
    return {
        'gamma': gamma,
        'delta': delta,
        'theoretical_bias': theoretical_bias,
        'actual_bias': actual_bias,
        'naive_estimate': naive_estimate,
        'adjusted_estimate': adjusted_estimate
    }

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def calculate_confounding_bias(df: pd.DataFrame, confound_var: str = 'X') -> dict:
    """计算混淆偏差的各个组成部分"""
    from sklearn.linear_model import LinearRegression
    
    # 朴素估计（不控制 X）
    model_naive = LinearRegression()
    model_naive.fit(df[['T']], df['Y'])
    naive_estimate = model_naive.coef_[0]
    
    # 调整估计（控制 X）
    model_adjusted = LinearRegression()
    model_adjusted.fit(df[['T', confound_var]], df['Y'])
    adjusted_estimate = model_adjusted.coef_[0]
    
    # gamma（X 对 Y 的效应，控制 T）
    gamma = model_adjusted.coef_[1]
    
    # delta（X 与 T 的关联）
    model_delta = LinearRegression()
    model_delta.fit(df[[confound_var]], df['T'])
    delta = model_delta.coef_[0]
    
    # 理论偏差 = gamma × delta
    theoretical_bias = gamma * delta
    
    # 实际偏差
    actual_bias = naive_estimate - adjusted_estimate
    
    return {
        'gamma': gamma,
        'delta': delta,
        'theoretical_bias': theoretical_bias,
        'actual_bias': actual_bias,
        'naive_estimate': naive_estimate,
        'adjusted_estimate': adjusted_estimate
    }
```

**解释**：
- **γ (gamma)**: X 对 Y 的偏回归系数，衡量控制 T 后 X 对 Y 的影响
- **δ (delta)**: X 对 T 的回归系数，衡量 X 和 T 的关联强度  
- **偏差公式**: Bias = γ × δ（遗漏变量偏差的经典公式）
- **验证**: theoretical_bias 应该非常接近 actual_bias
</details>

In [ ]:
# 测试你的代码
result = calculate_confounding_bias(df)

if result['gamma'] is not None:
    print("🔬 混淆偏差分解:")
    print("=" * 50)
    print(f"\n📐 偏差公式组成部分:")
    print(f"   γ (X 对 Y 的效应): {result['gamma']:.4f}")
    print(f"   δ (X 与 T 的关联): {result['delta']:.4f}")
    print(f"\n📊 偏差计算:")
    print(f"   理论偏差 (γ × δ): {result['theoretical_bias']:.4f}")
    print(f"   实际偏差: {result['actual_bias']:.4f}")
    print(f"\n📈 效应估计:")
    print(f"   朴素估计: {result['naive_estimate']:.4f}")
    print(f"   调整估计: {result['adjusted_estimate']:.4f}")
    print(f"   真实效应: {true_ate:.4f}")
    
    # 验证公式
    if abs(result['theoretical_bias'] - result['actual_bias']) < 0.1:
        print(f"\n✅ 公式验证成功！理论偏差 ≈ 实际偏差")
else:
    print("❌ 请完成 calculate_confounding_bias 函数！")

### 💡 关键洞察

从上面的结果可以看到：

1. **偏差的方向**：γ 和 δ 同号时，偏差为正（高估效应）
2. **偏差的大小**：取决于 γ 和 δ 的乘积
3. **公式的精确性**：理论偏差和实际偏差几乎完全一致！

---

## 📈 Part 2: 混淆强度实验

让我们系统地研究：混淆强度如何影响估计偏差？

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def experiment_confounding_strength(n: int = 2000, true_ate: float = 2.0,
                                   confounding_strengths: list = None, seed: int = 42):
    """实验不同混淆强度对估计的影响"""
    if confounding_strengths is None:
        confounding_strengths = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
    
    np.random.seed(seed)
    results = []
    
    for strength in confounding_strengths:
        X = np.random.randn(n)
        
        # 生成处理 T（受 X 影响，强度由 strength 控制）
        propensity = 1 / (1 + np.exp(-strength * X))
        T = np.random.binomial(1, propensity)
        
        # 生成结果 Y（受 T 和 X 影响）
        Y = 5 + true_ate * T + strength * X + np.random.randn(n) * 0.5
        
        df_temp = pd.DataFrame({'X': X, 'T': T, 'Y': Y})
        
        # 朴素估计
        naive_est = df_temp[df_temp['T']==1]['Y'].mean() - df_temp[df_temp['T']==0]['Y'].mean()
        
        # 调整估计
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()
        model.fit(df_temp[['T', 'X']], df_temp['Y'])
        adjusted_est = model.coef_[0]
        
        results.append({
            'confounding_strength': strength,
            'naive_estimate': naive_est,
            'adjusted_estimate': adjusted_est,
            'naive_bias': naive_est - true_ate,
            'adjusted_bias': adjusted_est - true_ate
        })
    
    return pd.DataFrame(results)
```

**解释**：
- **混淆强度参数化**: strength 同时控制 X→T 和 X→Y 的强度
- **观察**: 混淆强度越大，朴素估计偏差越大
- **稳健性**: 调整估计在各种强度下都接近真实值
</details>

In [ ]:
def experiment_confounding_strength(
    n: int = 2000,
    true_ate: float = 2.0,
    confounding_strengths: list = None,
    seed: int = 42
) -> pd.DataFrame:
    """
    实验不同混淆强度对估计的影响
    
    混淆强度同时影响:
    - P(T=1|X): 倾向得分
    - Y 中 X 的系数
    """
    if confounding_strengths is None:
        confounding_strengths = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
    
    np.random.seed(seed)
    results = []
    
    for strength in confounding_strengths:
        # 生成混淆数据
        X = np.random.randn(n)
        
        # TODO: 生成处理 T
        # P(T=1|X) = sigmoid(strength * X)
        propensity = 1 / (1 + np.exp(-strength * X))
        T = None  # 👈 你的代码: np.random.binomial(1, propensity)
        
        # TODO: 生成结果 Y
        # Y = 5 + true_ate * T + strength * X + noise
        Y = None  # 👈 你的代码
        
        if T is not None and Y is not None:
            df_temp = pd.DataFrame({'X': X, 'T': T, 'Y': Y})
            
            # 计算朴素估计
            naive_est = df_temp[df_temp['T']==1]['Y'].mean() - df_temp[df_temp['T']==0]['Y'].mean()
            
            # 计算调整估计
            model = LinearRegression()
            model.fit(df_temp[['T', 'X']], df_temp['Y'])
            adjusted_est = model.coef_[0]
            
            results.append({
                'confounding_strength': strength,
                'naive_estimate': naive_est,
                'adjusted_estimate': adjusted_est,
                'naive_bias': naive_est - true_ate,
                'adjusted_bias': adjusted_est - true_ate
            })
    
    return pd.DataFrame(results) if results else None

In [ ]:
# 运行实验
exp_results = experiment_confounding_strength(n=3000)

if exp_results is not None and not exp_results.empty:
    print("📊 混淆强度实验结果:")
    print("=" * 70)
    print(exp_results.round(3).to_string(index=False))
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 图1: 估计值 vs 混淆强度
    ax1 = axes[0]
    ax1.plot(exp_results['confounding_strength'], exp_results['naive_estimate'], 
             'o-', label='朴素估计', color='red', markersize=8)
    ax1.plot(exp_results['confounding_strength'], exp_results['adjusted_estimate'], 
             's-', label='调整估计', color='green', markersize=8)
    ax1.axhline(2.0, color='blue', linestyle='--', label='真实 ATE = 2', linewidth=2)
    ax1.set_xlabel('混淆强度', fontsize=12)
    ax1.set_ylabel('ATE 估计', fontsize=12)
    ax1.set_title('估计值随混淆强度的变化', fontsize=14)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # 图2: 偏差 vs 混淆强度
    ax2 = axes[1]
    ax2.bar(exp_results['confounding_strength'] - 0.1, exp_results['naive_bias'], 
            width=0.2, label='朴素估计偏差', color='red', alpha=0.7)
    ax2.bar(exp_results['confounding_strength'] + 0.1, exp_results['adjusted_bias'], 
            width=0.2, label='调整估计偏差', color='green', alpha=0.7)
    ax2.axhline(0, color='black', linestyle='-', linewidth=1)
    ax2.set_xlabel('混淆强度', fontsize=12)
    ax2.set_ylabel('偏差', fontsize=12)
    ax2.set_title('偏差随混淆强度的变化', fontsize=14)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 关键发现:")
    print("   1. 混淆强度为 0 时，朴素估计也是无偏的")
    print("   2. 混淆强度越大，朴素估计偏差越大")
    print("   3. 调整估计始终接近真实值！")
else:
    print("❌ 请完成 experiment_confounding_strength 函数！")

---

## 🎪 Part 3: Simpson's Paradox（辛普森悖论）

### 一个真实的故事

1973 年，伯克利大学被指控性别歧视：

| | 申请人 | 录取率 |
|--|--------|--------|
| 男性 | 8442 | 44% |
| 女性 | 4321 | 35% |

看起来确实歧视女性！

但仔细分析每个系的数据后，发现**大多数系的女性录取率反而更高**！

怎么回事？🤯

原来：
- 女性倾向于申请录取率低的「热门系」（如心理学）
- 男性倾向于申请录取率高的「冷门系」（如工程学）

「院系」是混淆变量！

In [ ]:
def create_simpson_paradox_data(n_per_group: int = 500, seed: int = 42) -> pd.DataFrame:
    """
    创建展示 Simpson's Paradox 的数据
    
    场景: 研究某药物对康复率的影响
    - 有两个医院 (A 和 B)
    - 医院 A 接收重症患者多，药物使用率高
    - 医院 B 接收轻症患者多，药物使用率低
    - 药物实际上有正效应！
    
    设计数据使得:
    - 整体: 用药组康复率 < 未用药组康复率（看起来药物有害!）
    - 分医院: 用药组康复率 > 未用药组康复率（药物实际有益）
    """
    np.random.seed(seed)
    data = []
    
    # TODO: 医院 A（重症多，用药多）
    # 重症基础康复率低（30%），用药提高到 50%
    # 大部分重症患者在这里，大部分接受治疗
    
    # 医院 A - 用药组
    n_A_treated = int(n_per_group * 1.5)  # 用药人数多
    # 👈 你的代码: 用药后康复率 50%
    recovery_A_treated = None  # np.random.binomial(1, 0.50, n_A_treated)
    
    for i in range(n_A_treated):
        if recovery_A_treated is not None:
            data.append({
                '医院': 'A (重症)',
                '用药': 1,
                '康复': recovery_A_treated[i],
                '病情': '重症'
            })
    
    # 医院 A - 未用药组
    n_A_control = int(n_per_group * 0.3)  # 未用药人数少
    # 👈 你的代码: 未用药康复率 30%
    recovery_A_control = None  # np.random.binomial(1, 0.30, n_A_control)
    
    for i in range(n_A_control):
        if recovery_A_control is not None:
            data.append({
                '医院': 'A (重症)',
                '用药': 0,
                '康复': recovery_A_control[i],
                '病情': '重症'
            })
    
    # TODO: 医院 B（轻症多，用药少）
    # 轻症基础康复率高（70%），用药提高到 90%
    
    # 医院 B - 用药组
    n_B_treated = int(n_per_group * 0.3)  # 用药人数少
    # 👈 你的代码: 用药后康复率 90%
    recovery_B_treated = None  # np.random.binomial(1, 0.90, n_B_treated)
    
    for i in range(n_B_treated):
        if recovery_B_treated is not None:
            data.append({
                '医院': 'B (轻症)',
                '用药': 1,
                '康复': recovery_B_treated[i],
                '病情': '轻症'
            })
    
    # 医院 B - 未用药组
    n_B_control = int(n_per_group * 1.5)  # 未用药人数多
    # 👈 你的代码: 未用药康复率 70%
    recovery_B_control = None  # np.random.binomial(1, 0.70, n_B_control)
    
    for i in range(n_B_control):
        if recovery_B_control is not None:
            data.append({
                '医院': 'B (轻症)',
                '用药': 0,
                '康复': recovery_B_control[i],
                '病情': '轻症'
            })
    
    return pd.DataFrame(data)

In [ ]:
def analyze_simpson_paradox(df: pd.DataFrame) -> dict:
    """
    分析 Simpson's Paradox
    """
    results = {}
    
    # TODO: 整体效应
    overall_treated = df[df['用药'] == 1]['康复'].mean()
    overall_control = df[df['用药'] == 0]['康复'].mean()
    results['整体-用药组康复率'] = None  # 👈 你的代码
    results['整体-未用药组康复率'] = None  # 👈 你的代码
    results['整体-效应'] = None  # 👈 你的代码: overall_treated - overall_control
    
    # TODO: 医院 A 效应
    df_A = df[df['医院'] == 'A (重症)']
    results['医院A-用药组康复率'] = None  # 👈 你的代码
    results['医院A-未用药组康复率'] = None  # 👈 你的代码
    results['医院A-效应'] = None  # 👈 你的代码
    
    # TODO: 医院 B 效应
    df_B = df[df['医院'] == 'B (轻症)']
    results['医院B-用药组康复率'] = None  # 👈 你的代码
    results['医院B-未用药组康复率'] = None  # 👈 你的代码
    results['医院B-效应'] = None  # 👈 你的代码
    
    return results

In [ ]:
# 创建并分析 Simpson's Paradox 数据
simpson_df = create_simpson_paradox_data(n_per_group=500)

if simpson_df is not None and not simpson_df.empty:
    print("📊 数据概览:")
    print(f"   总样本量: {len(simpson_df)}")
    print(f"\n各组人数:")
    print(simpson_df.groupby(['医院', '用药']).size().unstack())
    
    analysis = analyze_simpson_paradox(simpson_df)
    
    if analysis.get('整体-效应') is not None:
        print("\n" + "=" * 60)
        print("🎪 Simpson's Paradox 分析结果:")
        print("=" * 60)
        
        print(f"\n📈 整体分析（不分医院）:")
        print(f"   用药组康复率: {analysis['整体-用药组康复率']:.1%}")
        print(f"   未用药组康复率: {analysis['整体-未用药组康复率']:.1%}")
        print(f"   效应: {analysis['整体-效应']:+.1%}")
        if analysis['整体-效应'] < 0:
            print(f"   结论: 用药似乎有害！❌")
        
        print(f"\n🏥 医院 A（重症患者）:")
        print(f"   用药组康复率: {analysis['医院A-用药组康复率']:.1%}")
        print(f"   未用药组康复率: {analysis['医院A-未用药组康复率']:.1%}")
        print(f"   效应: {analysis['医院A-效应']:+.1%}")
        if analysis['医院A-效应'] > 0:
            print(f"   结论: 用药有效！✅")
        
        print(f"\n🏥 医院 B（轻症患者）:")
        print(f"   用药组康复率: {analysis['医院B-用药组康复率']:.1%}")
        print(f"   未用药组康复率: {analysis['医院B-未用药组康复率']:.1%}")
        print(f"   效应: {analysis['医院B-效应']:+.1%}")
        if analysis['医院B-效应'] > 0:
            print(f"   结论: 用药有效！✅")
        
        print(f"\n🎭 悖论解释:")
        print(f"   整体看起来用药有害，但分层后每个医院用药都有效！")
        print(f"   原因: 重症患者用药多，轻症患者不用药多")
        print(f"   '病情严重程度'是混淆变量！")
else:
    print("❌ 请完成 create_simpson_paradox_data 函数！")

In [ ]:
# 可视化 Simpson's Paradox
if simpson_df is not None and not simpson_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 图1: 整体对比
    ax1 = axes[0]
    overall_data = simpson_df.groupby('用药')['康复'].mean()
    colors = ['steelblue', 'coral']
    bars = ax1.bar(['未用药', '用药'], [overall_data[0], overall_data[1]], color=colors)
    ax1.set_ylabel('康复率', fontsize=12)
    ax1.set_title('整体康复率\n(看起来用药有害!)', fontsize=14)
    ax1.set_ylim(0, 1)
    for bar, val in zip(bars, [overall_data[0], overall_data[1]]):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{val:.1%}', ha='center', fontsize=12)
    
    # 图2: 分层对比
    ax2 = axes[1]
    stratified_data = simpson_df.groupby(['医院', '用药'])['康复'].mean().unstack()
    x = np.arange(2)
    width = 0.35
    bars1 = ax2.bar(x - width/2, stratified_data[0], width, label='未用药', color='steelblue')
    bars2 = ax2.bar(x + width/2, stratified_data[1], width, label='用药', color='coral')
    ax2.set_xticks(x)
    ax2.set_xticklabels(['医院A (重症)', '医院B (轻症)'])
    ax2.set_ylabel('康复率', fontsize=12)
    ax2.set_title('分层康复率\n(两个医院用药都有效!)', fontsize=14)
    ax2.legend()
    ax2.set_ylim(0, 1)
    
    for bars, col in [(bars1, 0), (bars2, 1)]:
        for bar, hospital in zip(bars, stratified_data.index):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                    f'{stratified_data.loc[hospital, col]:.1%}', ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

### 💡 Simpson's Paradox 的本质

Simpson's Paradox 不是真正的「悖论」，它告诉我们：

1. **整体趋势 ≠ 分层趋势**: 当存在混淆变量时，整体数据可能给出误导性的结论
2. **因果方向很重要**: 应该控制混淆变量（病情），而不是被它误导
3. **数据会说谎**: 没有正确的因果分析，数据可能告诉你完全相反的结论！

---

## 🔮 Part 4: 敏感性分析

### 当混淆变量不可观测时怎么办？

在现实中，我们经常无法观测到所有混淆变量。比如：

- 研究吸烟对肺癌的影响，但无法观测「遗传因素」
- 研究教育对收入的影响，但无法观测「天赋」

这时候，我们需要问：**如果存在未观测混淆，我们的结论还可靠吗？**

敏感性分析就是用来回答这个问题的！

In [ ]:
def sensitivity_to_unmeasured_confounding(
    df: pd.DataFrame,
    gamma_range: np.ndarray = None,
    delta_range: np.ndarray = None
) -> pd.DataFrame:
    """
    敏感性分析: 如果存在未观测混淆，估计会如何变化？
    
    假设存在未观测混淆变量 U:
    - U 对 Y 的效应为 gamma_u
    - U 与 T 的关联为 delta_u
    - 偏差 = gamma_u * delta_u
    
    对不同的 (gamma_u, delta_u) 组合，计算可能的真实效应
    """
    if gamma_range is None:
        gamma_range = np.linspace(-2, 2, 9)
    if delta_range is None:
        delta_range = np.linspace(-1, 1, 9)
    
    # 当前估计（假设已控制观测到的混淆）
    model = LinearRegression()
    model.fit(df[['T', 'X']], df['Y'])
    current_estimate = model.coef_[0]
    
    results = []
    for gamma_u in gamma_range:
        for delta_u in delta_range:
            # 可能的偏差（如果存在未观测混淆 U）
            possible_bias = gamma_u * delta_u
            # 可能的真实效应
            possible_true_effect = current_estimate - possible_bias
            
            results.append({
                'gamma_u': gamma_u,
                'delta_u': delta_u,
                'possible_bias': possible_bias,
                'possible_true_effect': possible_true_effect
            })
    
    return pd.DataFrame(results)

In [ ]:
# 运行敏感性分析
sensitivity = sensitivity_to_unmeasured_confounding(df)

print("🔮 敏感性分析结果:")
print("=" * 50)

# 当前估计
model = LinearRegression()
model.fit(df[['T', 'X']], df['Y'])
current_est = model.coef_[0]

print(f"当前估计（控制 X 后）: {current_est:.4f}")
print(f"\n如果存在未观测混淆 U:")
print(f"   可能的真实效应范围: [{sensitivity['possible_true_effect'].min():.2f}, {sensitivity['possible_true_effect'].max():.2f}]")

# 找出使效应变为 0 或负数的条件
zero_effect = sensitivity[sensitivity['possible_true_effect'] < 0]
if len(zero_effect) > 0:
    print(f"\n⚠️ 以下情况会使效应变为负数:")
    print(f"   需要 γ_u × δ_u > {current_est:.2f}")
    print(f"   例如: γ_u = 2, δ_u = 1.5 (强混淆)")

In [ ]:
# 可视化敏感性分析
pivot = sensitivity.pivot_table(
    index='gamma_u', 
    columns='delta_u', 
    values='possible_true_effect'
)

plt.figure(figsize=(10, 8))
sns.heatmap(
    pivot, 
    annot=True, 
    fmt='.2f', 
    cmap='RdYlGn',
    center=0,
    vmin=-2,
    vmax=4
)
plt.xlabel('δ_u (U 与 T 的关联)', fontsize=12)
plt.ylabel('γ_u (U 对 Y 的效应)', fontsize=12)
plt.title('敏感性分析: 可能的真实效应\n(红色 = 负效应, 绿色 = 正效应)', fontsize=14)
plt.tight_layout()
plt.show()

print("\n📊 热力图解读:")
print("   - 中心 (0, 0): 无未观测混淆，效应 = 当前估计")
print("   - 绿色区域: 即使有未观测混淆，效应仍为正")
print("   - 红色区域: 未观测混淆可能使效应为负")
print("   - 效应为 0 的边界: γ_u × δ_u = 当前估计")

---

## 📝 Part 5: 思考题

### 问题 1: 为什么 Simpson's Paradox 不是真正的悖论？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 2: 偏差公式 Bias = γ × δ 中，什么情况下偏差为 0？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 在实践中，如何判断是否存在混淆？

**提示:** 想想领域知识、敏感性分析、阴性对照...

**你的答案:**

*（在这里写下你的思考...）*

---

---

## 🎯 面试题模拟

### 概念题

**Q1: 解释遗漏变量偏差（OVB）公式 Bias = γ × δ，并说明什么情况下偏差为零。**

<details>
<summary>答案</summary>

**OVB 公式**: Bias = γ × δ

- **γ (gamma)**: 遗漏变量 X 对结果 Y 的效应（控制 T 后）
- **δ (delta)**: 遗漏变量 X 与处理 T 的关联

**偏差为零的情况**:

1. **γ = 0**: X 对 Y 没有直接影响
   - 即使 X 与 T 相关，遗漏 X 也不会造成偏差
   - 例子：X 只影响 T，不影响 Y

2. **δ = 0**: X 与 T 不相关（独立）
   - 即使 X 影响 Y，遗漏 X 也不会造成偏差
   - 例子：随机实验中，T 独立于所有协变量

**关键洞察**: 
- 只有当 X 同时满足两个条件时才会产生偏差：
  1. X 影响 Y（γ ≠ 0）
  2. X 与 T 相关（δ ≠ 0）
- 这正是混淆变量的定义！

**实践意义**:
- 不是所有遗漏变量都会造成偏差
- 只需控制同时影响 T 和 Y 的变量
- 这是后门准则的理论基础

</details>

---

**Q2: Simpson's Paradox 在因果推断中的意义是什么？它告诉我们什么？**

<details>
<summary>答案</summary>

**Simpson's Paradox**: 整体趋势和分层趋势可能相反的现象

**核心教训**:

1. **聚合数据会误导**: 不分层的整体数据可能给出错误结论
2. **混淆是罪魁祸首**: 悖论的根源是存在混淆变量
3. **因果方向很重要**: 需要知道变量之间的因果关系才能正确分析

**经典例子**: 

伯克利性别歧视案（1973）:
- 整体：男性录取率 44%，女性 35% → 看起来歧视女性
- 分院系：大多数院系女性录取率更高或相同
- 原因：女性倾向申请竞争更激烈的院系

**判断标准**: 应该分层还是聚合？

关键在于因果结构：
- 如果院系是**混淆变量**（影响性别分布和录取率）→ 应该分层
- 如果院系是**碰撞变量**或**中介** → 不应该分层

**一般原则**:
1. 先画因果图
2. 应用后门准则
3. 控制混淆变量，不控制中介/碰撞变量

**实践警示**:
- 不要盲目相信聚合统计
- 也不要盲目分层（可能引入碰撞偏差）
- 需要领域知识和因果思维

</details>

---

**Q3: 什么是敏感性分析？为什么在因果推断中很重要？**

<details>
<summary>答案</summary>

**敏感性分析**: 评估估计结果对未观测混淆的稳健性

**核心思想**: 
- 我们永远无法确定是否观测到了所有混淆变量
- 敏感性分析问："如果存在未观测混淆 U，结论会改变吗？"

**方法**:

1. **参数化未观测混淆**:
   - γ_U: U 对 Y 的影响
   - δ_U: U 与 T 的关联

2. **计算可能的偏差范围**:
   - 真实效应 ∈ [估计值 - γ_U×δ_U, 估计值 + γ_U×δ_U]

3. **评估稳健性**:
   - 需要多强的混淆才能推翻结论？
   - 这样的混淆合理吗？

**Rosenbaum Bounds 例子**:

假设当前估计 ATE = 5，显著性 p < 0.05

| Γ值 | 含义 | p值范围 | 结论 |
|-----|------|---------|------|
| 1.0 | 无未观测混淆 | [0.01, 0.01] | 显著 |
| 1.5 | 倾向得分可相差50% | [0.01, 0.08] | 仍显著 |
| 2.0 | 倾向得分可相差100% | [0.01, 0.15] | 不稳健 |

解读：
- Γ = 1.5 时结论仍显著 → 结果相对稳健
- Γ = 2.0 时结论可能不显著 → 存在风险

**实践价值**:

1. **诚实性**: 承认不确定性，不过度自信
2. **说服力**: 展示结果对假设的依赖程度
3. **指导收集数据**: 指出哪些未观测变量最关键

**何时使用**:
- 观测研究（非随机）
- 已控制已知混淆，但担心遗漏
- 需要向利益相关者证明结论的稳健性

</details>

---

### 编程题

**Q4: 实现一个函数，模拟 Simpson's Paradox，展示整体和分层的效应估计。**

<details>
<summary>参考代码</summary>

```python
def create_simpsons_paradox_example(n_per_group=500, seed=42):
    """
    创建 Simpson's Paradox 示例
    
    场景：评估新药效果
    - 两个医院 A 和 B
    - 医院 A 主要收治重症（用药多）
    - 医院 B 主要收治轻症（用药少）
    - 每个医院内用药都有正效应
    - 但整体看起来用药效果为负！
    """
    np.random.seed(seed)
    data = []
    
    # 医院 A（重症为主）
    # 基础康复率低，但用药提高康复率
    n_A_treated = int(n_per_group * 1.6)  # 医院A用药多
    n_A_control = int(n_per_group * 0.4)
    
    recovery_A_treated = np.random.binomial(1, 0.50, n_A_treated)  # 用药康复率50%
    recovery_A_control = np.random.binomial(1, 0.30, n_A_control)  # 不用药30%
    
    for r in recovery_A_treated:
        data.append({'医院': 'A(重症)', '用药': 1, '康复': r})
    for r in recovery_A_control:
        data.append({'医院': 'A(重症)', '用药': 0, '康复': r})
    
    # 医院 B（轻症为主）
    # 基础康复率高，用药进一步提高
    n_B_treated = int(n_per_group * 0.4)  # 医院B用药少
    n_B_control = int(n_per_group * 1.6)
    
    recovery_B_treated = np.random.binomial(1, 0.90, n_B_treated)  # 用药康复率90%
    recovery_B_control = np.random.binomial(1, 0.70, n_B_control)  # 不用药70%
    
    for r in recovery_B_treated:
        data.append({'医院': 'B(轻症)', '用药': 1, '康复': r})
    for r in recovery_B_control:
        data.append({'医院': 'B(轻症)', '用药': 0, '康复': r})
    
    df = pd.DataFrame(data)
    
    # 计算效应
    # 整体效应（错误的！）
    overall_treated = df[df['用药']==1]['康复'].mean()
    overall_control = df[df['用药']==0]['康复'].mean()
    overall_effect = overall_treated - overall_control
    
    # 分层效应（正确的）
    df_A = df[df['医院']=='A(重症)']
    effect_A = df_A[df_A['用药']==1]['康复'].mean() - df_A[df_A['用药']==0]['康复'].mean()
    
    df_B = df[df['医院']=='B(轻症)']
    effect_B = df_B[df_B['用药']==1]['康复'].mean() - df_B[df_B['用药']==0]['康复'].mean()
    
    # 标准化效应（按总体医院分布加权）
    weight_A = (len(df_A) / len(df))
    weight_B = (len(df_B) / len(df))
    standardized_effect = effect_A * weight_A + effect_B * weight_B
    
    print(f"📊 Simpson's Paradox 示例分析")
    print(f"=" * 60)
    print(f"\n整体效应（不分层）: {overall_effect:+.1%}")
    print(f"  用药组康复率: {overall_treated:.1%}")
    print(f"  对照组康复率: {overall_control:.1%}")
    if overall_effect < 0:
        print(f"  ❌ 错误结论：用药有害！")
    
    print(f"\n分层效应:")
    print(f"  医院A（重症）: {effect_A:+.1%} ✅")
    print(f"  医院B（轻症）: {effect_B:+.1%} ✅")
    print(f"  ✅ 正确结论：每个医院内用药都有益！")
    
    print(f"\n标准化效应（正确加权）: {standardized_effect:+.1%}")
    print(f"\n🔍 悖论原因:")
    print(f"  - 医院A（重症）用药比例高: {df_A['用药'].mean():.1%}")
    print(f"  - 医院B（轻症）用药比例低: {df_B['用药'].mean():.1%}")
    print(f"  - 用药组混入了更多重症患者，拉低了整体康复率")
    
    return df

# 运行示例
df_simpson = create_simpsons_paradox_example()
```

**关键点**:
- 整体效应被混淆变量（医院/病情）误导
- 每个层内效应都是正的，但整体可能是负的
- 必须按正确权重标准化

</details>

---

**Q5: 实现敏感性分析函数，评估估计对未观测混淆的稳健性。**

<details>
<summary>参考代码（已在notebook中实现）</summary>

见 notebook 中的 `sensitivity_to_unmeasured_confounding` 函数。

**使用示例**:

```python
# 假设当前估计 ATE = 5（控制了观测到的 X）
# 评估：如果存在未观测混淆 U，效应会在什么范围？

gamma_range = np.linspace(-3, 3, 13)  # U 对 Y 的可能影响
delta_range = np.linspace(-1, 1, 11)  # U 与 T 的可能关联

sensitivity = sensitivity_to_unmeasured_confounding(
    df, gamma_range=gamma_range, delta_range=delta_range
)

# 查看在什么情况下效应会变号
print("使效应变为负数的情况:")
negative_cases = sensitivity[sensitivity['possible_true_effect'] < 0]
print(negative_cases[['gamma_u', 'delta_u', 'possible_true_effect']])

# 判断稳健性
if len(negative_cases) > 0:
    min_gamma = negative_cases['gamma_u'].abs().min()
    min_delta = negative_cases['delta_u'].abs().min()
    print(f"\n需要 |γ_U| ≥ {min_gamma:.2f} 且 |δ_U| ≥ {min_delta:.2f} 才能推翻结论")
    print(f"判断：这样的未观测混淆是否合理？")
else:
    print(f"\n在给定参数范围内，效应始终为正")
    print(f"结论较稳健")
```

**实践技巧**:
1. 根据领域知识设定合理的 γ 和 δ 范围
2. 可视化敏感性热图（如notebook中所示）
3. 关注使结论发生质变的临界值
4. 与已观测混淆的影响对比，评估合理性

</details>

---

---

## 📐 数学推导

### 1. 遗漏变量偏差公式（OVB）

**场景**: 真实模型为 $Y = \beta_0 + \beta_T T + \beta_X X + \varepsilon$，但我们遗漏了 X，只估计 $Y = \alpha_0 + \alpha_T T + u$

**定理**: 遗漏 X 导致的偏差为：

$$\text{Bias}(\hat{\alpha}_T) = E[\hat{\alpha}_T] - \beta_T = \beta_X \cdot \delta$$

其中 $\delta$ 是 $T$ 对 $X$ 回归的系数：$X = \gamma_0 + \delta T + v$

**证明**:

1. 真实模型：
   $$Y = \beta_0 + \beta_T T + \beta_X X + \varepsilon$$

2. 将 $X = \gamma_0 + \delta T + v$ 代入：
   $$Y = \beta_0 + \beta_T T + \beta_X(\gamma_0 + \delta T + v) + \varepsilon$$
   $$= (\beta_0 + \beta_X \gamma_0) + (\beta_T + \beta_X \delta) T + (\beta_X v + \varepsilon)$$

3. 简化模型的系数：
   $$\alpha_T = \beta_T + \beta_X \delta$$

4. 因此偏差：
   $$\text{Bias} = \alpha_T - \beta_T = \beta_X \delta$$

**符号说明**:
- $\beta_X$ (gamma): X 对 Y 的真实效应
- $\delta$: X 与 T 的关联（T 对 X 回归的系数，或 X 对 T 回归的系数乘以方差比）

**偏差方向**:
| $\beta_X$ | $\delta$ | 偏差 | 解释 |
|----------|---------|------|------|
| + | + | + | 高估 |
| + | - | - | 低估 |
| - | + | - | 低估 |
| - | - | + | 高估 |

---

### 2. 选择偏差的数学表达

**定义**: 当处理分配与潜在结果相关时产生的偏差。

$$\begin{align}
E[Y|T=1] - E[Y|T=0] &= E[Y(1)|T=1] - E[Y(0)|T=0] \\
&= \underbrace{E[Y(1) - Y(0)|T=1]}_{\text{ATT（真实效应）}} \\
&\quad + \underbrace{E[Y(0)|T=1] - E[Y(0)|T=0]}_{\text{选择偏差}}
\end{align}$$

**选择偏差项**: $E[Y(0)|T=1] - E[Y(0)|T=0]$

**解释**: 即使不接受处理，处理组和对照组的结果也不同。

**例子**:
- T = 上大学，Y = 收入
- 选择偏差 = "上大学的人即使不上大学收入也会更高"
- 原因：上大学的人可能本来就更聪明、更努力

**消除方法**:
1. 随机化：使 $E[Y(0)|T=1] = E[Y(0)|T=0]$
2. 条件独立假设：控制所有影响选择的协变量 X
3. 工具变量、DID 等准实验方法

---

### 3. Simpson's Paradox 的概率论解释

**现象**: 整体趋势和分层趋势相反

**数学条件**: 存在变量 Z 使得：

$$\text{sign}\left(\frac{P(Y=1|T=1)}{P(Y=1|T=0)}\right) \neq \text{sign}\left(\frac{P(Y=1|T=1, Z=z)}{P(Y=1|T=0, Z=z)}\right)$$

对所有 z 成立。

**例子**: 医院数据

|  | 用药康复率 | 未用药康复率 | 用药更好？ |
|--|-----------|-------------|-----------|
| 整体 | 40% | 50% | ✗ |
| 医院A（重症） | 50% | 30% | ✓ |
| 医院B（轻症） | 90% | 70% | ✓ |

**数学验证**:

设：
- $n_{A,1} = 200$ (医院A用药), $n_{A,0} = 100$ (医院A不用药)
- $n_{B,1} = 100$ (医院B用药), $n_{B,0} = 200$ (医院B不用药)

整体康复率：
$$P(Y=1|T=1) = \frac{200 \times 0.5 + 100 \times 0.9}{300} = \frac{190}{300} \approx 0.63$$

但这**错误地**混合了不同严重程度的患者！

正确方法（标准化）：
$$P(Y(1)=1) = P(Y=1|T=1, Z=A) \cdot P(Z=A) + P(Y=1|T=1, Z=B) \cdot P(Z=B)$$

使用**总体**的医院分布作为权重，而非用药组的分布。

---

### 4. 敏感性分析的数学框架

**问题**: 如果存在未观测混淆 U，估计会如何变化？

**模型**:

$$Y = \beta_0 + \beta_T T + \beta_X X + \beta_U U + \varepsilon$$

其中 U 未观测。

**当前估计**（控制 X 后）:

$$\hat{\beta}_T^{obs} = \beta_T + \beta_U \delta_U$$

其中 $\delta_U$ 是 U 对 T 回归时的系数（控制 X 后）。

**敏感性参数**:
- $\Gamma$ = 最大倾向得分比值：$\frac{P(T=1|X,U)/(1-P(T=1|X,U))}{P(T=1|X)/(1-P(T=1|X))}$
- $\Lambda$ = U 对 Y 的最大影响

**Rosenbaum bounds**: 
$$\beta_T \in [\hat{\beta}_T^{obs} - \beta_U \delta_U, \hat{\beta}_T^{obs} + \beta_U \delta_U]$$

**实践应用**:
- 如果 $\Gamma = 1.5$（倾向得分最多相差 50%），结论仍然显著 → 结果稳健
- 如果 $\Gamma = 2$ 就会改变结论 → 结果不稳健

---

### 问题 4: 实际案例分析

**场景**: 评估促销邮件对购买的影响

- T: 是否收到促销邮件
- Y: 是否购买

请分析:
1. 列出可能的混淆变量
2. 每个混淆变量如何同时影响 T 和 Y？
3. 朴素估计会高估还是低估真实效应？
4. 如何控制这些混淆？

**你的分析:**

*（在这里写下你的分析...）*

---

---

## 🎉 总结

### 核心公式

$$\text{偏差} = \gamma \times \delta = \text{(X对Y的效应)} \times \text{(X与T的关联)}$$

### 关键知识点

| 概念 | 定义 | 启示 |
|-----|------|------|
| 混淆偏差 | 遗漏混淆变量导致的估计偏差 | 可以精确计算！ |
| Simpson's Paradox | 整体趋势与分层趋势相反 | 不是悖论，是混淆 |
| 敏感性分析 | 评估未观测混淆的影响 | 即使有遗漏，也能评估 |

### 实践建议

1. **画因果图**: 在分析前先画出假设的因果结构
2. **识别混淆**: 找出同时影响处理和结果的变量
3. **做敏感性分析**: 评估结论对未观测混淆的稳健性
4. **谨慎下结论**: 除非有强有力的证据，否则保持谨慎

### 下一步

现在我们深刻理解了混淆偏差，接下来我们将学习具体的**处理方法**——倾向得分匹配 (PSM)！

---

**「知道偏差有多大，比不知道偏差存在要好得多。」**